# Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization

# Load Images

In [ ]:
path = './fer_images.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
image = tf.keras.preprocessing.image.load_img('./fer2013/train/Angry/1003.jpg')
image

# Train Test Split

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255,
                                     rotation_range=7,
                                     horizontal_flip=True,
                                     zoom_range=0.2)

train_dataset = train_generator.flow_from_directory('./fer2013/train',
                                                    target_size=(48, 48),
                                                    batch_size=16,
                                                    class_mode='categorical',
                                                    shuffle=True)

In [ ]:
train_dataset.classes

In [ ]:
np.unique(train_dataset.classes, return_counts=True) #count of each classes

In [ ]:
train_dataset.class_indices #label of each classes

In [ ]:
sns.countplot(x=train_dataset.classes);

In [ ]:
test_generator = ImageDataGenerator(rescale=1./255)

test_dataset = train_generator.flow_from_directory('./fer2013/validation',
                                                    target_size=(48, 48),
                                                    batch_size=1,
                                                    class_mode='categorical',
                                                    shuffle=False)

# Building CNN

In [ ]:
num_detectors = 32
num_classes = 7
width, height = 48, 48
epochs = 70

network = Sequential()

network.add(Conv2D(num_detectors, (3,3), activation='relu', padding = 'same', input_shape = (width, height, 3)))
network.add(BatchNormalization())
network.add(Conv2D(num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Flatten())

network.add(Dense(2 * num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(2 * num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(num_classes, activation='softmax'))
print(network.summary())

In [ ]:
network.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
network.fit(train_dataset, epochs=epochs)

# Save and Load Model

In [ ]:
model_json = network.to_json()
with open('network_emotions.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
from keras.models import save_model
network_saved = save_model(network, './content/weights_emotions.hdf5')

In [ ]:
with open('./network_emotions.json', 'r') as json_file:
    json_saved_model = json_file.read()
json_saved_model

In [ ]:
network_loaded = tf.keras.models.model_from_json(json_saved_model)
network_loaded.load_weights('./weights_emotions.hdf5')
network_loaded.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
network_loaded.summary()

# Evaluate Model

In [ ]:
network_loaded.evaluate(test_dataset)

In [ ]:
predictions = network_loaded.predict(test_dataset)
predictions

In [ ]:
predictions = np.argmax(predictions, axis=1)
predictions

In [ ]:
test_dataset.classes

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_dataset.classes, predictions)

In [ ]:
test_dataset.class_indices

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_dataset.classes, predictions)
cm

In [ ]:
sns.heatmap(cm, annot=True);

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_dataset.classes, predictions))

# Classify one image

In [ ]:
image = cv2.imread('./gabriel.png')
tf.keras.preprocessing.image.load_img('./gabriel.png')

In [ ]:
face_detector = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

In [ ]:
original_image = image.copy()
faces = face_detector.detectMultiScale(original_image)
faces

In [ ]:
#roi = region_of_image
roi = image[40:40+128, 162:162+128]
tf.keras.preprocessing.image.array_to_img(roi)

In [ ]:
roi.shape

In [ ]:
roi = cv2.resize(roi, (48, 48))
tf.keras.preprocessing.image.array_to_img(roi)

In [ ]:
roi = roi / 255 #normalize

In [ ]:
roi.shape

In [ ]:
roi = np.expand_dims(roi, axis=0)
roi.shape

In [ ]:
probs = network_loaded.predict(roi)
probs

In [ ]:
result = np.argmax(probs)
result, test_dataset.class_indices['Happy']

# Classify multiple images

In [ ]:
image = cv2.imread('./faces_emotions.png')
tf.keras.preprocessing.image.load_img('./faces_emotions.png')

In [ ]:
faces = face_detector.detectMultiScale(image)
faces

In [ ]:
emotions = list(test_dataset.class_indices.keys())
emotions

In [ ]:
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
tf.keras.preprocessing.image.array_to_img(image)

In [ ]:
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    roi = image[y:y+h, x:x+w]
    roi = cv2.resize(roi, (48,48))
    roi = roi / 255
    roi = np.expand_dims(roi, axis=0)
    prediction = network_loaded.predict(roi)
    result = np.argmax(prediction)
    cv2.putText(image, emotions[result], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
tf.keras.preprocessing.image.array_to_img(image)

# Classify in Videos

In [ ]:
cap = cv2.VideoCapture('./emotion_test06.MOV')
connected, video = cap.read()
print(connected, video.shape)

In [ ]:
save_path = './emotion_test06_result.MOV'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
output_video = cv2.VideoWriter(save_path, fourcc, fps, (video.shape[1], video.shape[0]))

In [ ]:
while(cv2.waitKey(1) < 0):
    connected, frame = cap.read()
    if not connected:
        break
    faces = face_detector.detectMultiScale(frame, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))
    if(len(faces)>0):
        for(x,y,w,h) in faces:
            frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            roi = frame[y:y+h, x:x+w]
            roi = cv2.resize(roi, (48,48))
            roi = roi / 255
            roi = np.expand_dims(roi, axis=0)
            prediction = network_loaded.predict(roi)

            if(prediction is not None):
                result=np.argmax(prediction)
                frame = cv2.putText(frame, emotions[result], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
                display(tf.keras.preprocessing.image.array_to_img(frame))
                output_video.write(frame)
output_video.release()
cv2.destroyAllWindows()